In [1]:
import json, os
from openai import OpenAI

source_data_path = "../data/raw_data/"
# filename = "test.json"
# filename = "summary_hin_India_election_2024_100.json"
# filename = "summary_eng_India_election_2024_100.json"
# filename = "summary_eng_Gaza_100.json"
filename = "summary_zho_Gaza_100.json"
# filename = "summary_zho_Territorial_disputes_in_the_South_China_Sea_100.json"
# filename = "summary_eng_Territorial_disputes_in_the_South_China_Sea_100.json"
source_filename = source_data_path + filename

des_data_path = "../data/QnA_data/"
des_filename = des_data_path + "v2_auto_QnA_" + filename

# Load articles
with open(source_filename, 'r', encoding='utf-8') as f:
    articles = json.load(f)

# Load questions
with open(des_data_path+'formated_output_questions_v2.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

# Set API key
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)


In [2]:
def get_answers(article, questions, model):
    article_title = article.get('title')
    article_content = article.get('body')
    print(f"Proceeding article {article_title}")
    # answers = {}
    # for q_id, question in questions.items():
    
    pre = "You are an AI trained to understand articles and generate concise answers to multiple-choice questions based on the content. \
        Please read the following article carefully. After reading, answer ALL the questions listed below. \
            Your answers must be in capital letters and formatted as a single string, where each question number is followed by its corresponding answer letter. \
                Separate each question-answer pair with a semicolon. \
                    Example format: 1A;2B;3C;4D;... \n\n"

    prompt = pre + f"Article Content:\n{article_content}\n\n Questions: {questions}\n"
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        # content = response.choices[0].message.content
        # answers[q_id] = content
    except Exception as e:
        print(f"Error: {e}")
        # answers[q_id] = "Sorry, error from GPT."
    return response

In [3]:
def convert_string_to_dict(answer_string):
    if answer_string == "":
        return {}
    if answer_string[-1] == ';':
        answer_string = answer_string[:-1]
    return {item[:-1]: item[-1] for item in answer_string.split(';')}

results = []
iteration = 0
max_iteration = len(articles)

for article in articles:
    # article_content = article['body']

    answers = get_answers(article, questions, model="gpt-4-turbo")

    pure_answers = answers.choices[0].message.content
    print(pure_answers)

    # convert string to dictionary 1A;2B;3C;44D to {'1': 'A', '2': 'B', '3': 'C', '44': 'D'}
    processed_answers = convert_string_to_dict(pure_answers)
    article['answers'] = processed_answers

    results.append(article)
    iteration += 1
    print(f"Processed {iteration} out of {max_iteration}")


# Save results to a JSON file
with open(des_filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

Proceeding article 美國與以色列為何認為可以用核武抹除國家種族滅絕？從美國參議員眾議員以色列部長及解放軍少將的核武言論談起
1B;2D;3B;4E;5C;6E;7E;8C;9E;10E;11E;12E;13E;14E;15A;16E;17E;18C;19E;20E;21E;22B;23C;24A;25D;26A;27C;28C;29B;30E;31C;32E;33B;34A;35E;36A;37E;38C;39E;40B;41E;42E;43A;44B;45E;46E;47A;48E;49E;50B;51E;52E
Processed 1 out of 100
Proceeding article 美國與以色列為何認為可以用核武抹除國家種族滅絕？從美國參議員眾議員以色列部長及解放軍少將的核武言論談起
1B;2D;3B;4E;5C;6E;7E;8C;9E;10E;11E;12E;13E;14E;15A;16E;17E;18C;19E;20E;21E;22B;23C;24A;25D;26A;27C;28C;29B;30E;31C;32E;33B;34A;35E;36A;37E;38C;39E;40B;41E;42E;43A;44B;45E;46E;47A;48E;49E;50B;51E;52E
Processed 2 out of 100
Proceeding article 以色列擊斃真主黨指揮官 遭數十枚火箭襲擊 | 黎巴嫩 | 新唐人电视台
1B;2E;3B;4E;5E;6E;7E;8A;9E;10E;11E;12E;13E;14E;15A;16E;17E;18E;19E;20E;21E;22E;23E;24A;25D;26A;27C;28A;29D;30E;31E;32E;33C;34A;35E;36E;37D;38E;39E;40E;41B;42E;43E;44E;45E;46E;47E;48E;49E;50B;51E;52E
Processed 3 out of 100
Proceeding article 【專文】當前加薩戰爭的人道和公義
1B;2D;3B;4B;5C;6E;7E;8C;9D;10E;11E;12B;13E;14E;15B;16E;17E;18C;19E;20E;21E;22B;23C;24A;25D;26E;27C;28